# Physikalisch sinvolle Randbedingungen und andere Konstanten

Für die Simulationen der Fußbodenheizungen werden diverse Natur-, Material- und andere Konstanten sowie Randbedingungen benötigt, um die (Anfangs-)Randwertprobleme richtig formululieren und lösen zu können.

Diese sollten entsprechend diskutiert, dokumentiert und/oder belegbar sein.

Zu dem Zweck kann (vorerst) dieses Jupyter-Notebook dienen, um tatsächliche Konstanten für den Code und die Quellen/Diskussionen an einem Ort zu verbinden.

## Schnittstelle

Ich schlage als Schnittstelle das JSON-Format vor, was menschlich lesbare Dateien aus Python-Datentypen erzeugen und wieder einlesen kann. So sind auch portable "Szenarien" denkbar, die außerhalb des Solvers erstellt werden können.

In [1]:
import json
import numpy as np
#import dolfin as dlfn

## Wärmeleitfähigkeiten

Die für die Simulation benötigten Wärmeleitfähigkeiten wurden aus den folgenden Quellen übernommen:

    "copper": dlfn.Constant(384),
    "screed": dlfn.Constant(1.4),
    "concrete": dlfn.Constant(2.1),
- **Eiche:** https://www.schweizer-fn.de/stoff/wleit_isolierung/wleit_isolierung.php
- **Styropor:** https://www.energieheld.de/daemmung/daemmstoffe/styropor-eps#eigenschaften
- **PUR/PIR Platten Alukaschiert:** https://www.baudiscount-paderborn.de/PUR-/-PIR-Daemmung-WLG-023-alukaschiert
    "cork": dlfn.Constant(0.05),

In [2]:
thermal_conductivities = {
    "copper"            : 384, # Kupfer
    "screed"            : 1.4, # Estrich
    "concrete"          : 2.1, # Beton
    "wood"              : 0.2, # Eiche:
    "polystyrene"       : 0.032, # Styropor
    "pur_alukaschiert"  : 0.023, # PUR/PIR Platten Alukaschiert
    "cork"              : 0.05, # Kork
}

with open('thermal_conductivities.json', 'w') as file:
    print(json.dumps(thermal_conductivities))
    json.dump(thermal_conductivities, file, ensure_ascii=False)


{"copper": 384, "screed": 1.4, "concrete": 2.1, "wood": 0.2, "polystyrene": 0.032, "pur_alukaschiert": 0.023, "cork": 0.05}


## Verlegeabstand und Rohrdurchmesser


"Ein „dickes“ Rohr mit 17 mm Außendurchmesser bietet dem Estrich allerdings mehr Fläche zur Wärmeübernahme pro laufenden Meter (533,8 cm²) als ein „dünnes“ Rohr mit nur 12 mm (376,8 cm²). Bei größeren Rohrdurchmessern könnte man möglicherweise den Verlegabstand verringern. Bei Fußbodenheizungen mit Wärmepumpe werden „dicke“ Rohrleitungen empfohlen, da hier mit niedrigen Vorlauftemperaturen gearbeitet wird. Ansonsten fällt bei Fußbodenheizungen mit Wärmepumpe der Verlegeabstand, abhängig vom Wärmebedarf des Raumes, in der Regel geringer aus, als bei anderen Wärmeerzeugern. Die Empfehlung liegt hier bei einem Verlegeabstand von 10 cm.

Fußbodenheizungen mit anderen Wärmeerzeugern, wie Gas, Holz, Pellets oder Öl, arbeiten mit höheren Vorlauftemperaturen. Der Verlegeabstand kann hier größer gewählt werden. Empfohlen wird hier ein Verlegeabstand von 15 cm – je nach Wärmebedarf des Raumes."

**Letzendlich können für Außendurchmesser der Heizrohre also Werte im Bereich von 12 bis 17 mm angenommen werden.**

**Für die Verlegeabstände liegen empfohlene Werte zwischen 10 und 15 cm.**


Quelle: https://www.heizsparer.de/heizung/heizkorper/fussbodenheizung/fussbodenheizung-planen-heizkreise-berechnen

## Materialstärken

- Trittschall- und Wärmedämmung: 15 mm - 50 mm
- Rohrstärke: 16 mm
- Fließestrich: 35 mm
- Zementestrich: 45 mm

führt zu:
- Min. Aufbauhöhe mit Fließestrich: 66 mm - 96 mm
- Min. Aufbauhöhe mit Zementestrich: 76 mm - 106 mm

Gesamtaufbauhöhe = Dämmstärke Systemplatte + Rohrstärke + Rohrüberdeckung


### Bodenbeläge

"Der Wert fließt laut Gebäude-Energie-Gesetz (GEG) sogar gesetzlich in die Berechnung des Energiebedarfs eines Gebäudes mit ein. Deshalb ist auch die Berechnung des Wärmedurchlasswiderstands in der ISO 6946:2005-06 Abschnitt 6.2 genormt:

Der Wärmedurchlasswiderstand (R) errechnet sich aus dem Quotienten der Dicke (d) und der Wärmeleitfähigkeit (ƛ – Lambda) des Materials.

Bei Bodenbelägen die aus mehreren unterschiedlichen Materialien oder Schichten bestehen, addieren sich deren Wärmedurchlasswiderstände."

\begin{table}[]
\begin{tabular}{lll}
\textbf{Material} & \textbf{Stärke} & \textbf{Wärmedurchlass in m²K/W} \\
Keramikfliesen    & 13mm            & 0,012                            \\
Korkparkett       & 11mm            & 0,129                            \\
Kunststoff-PVC    & 2mm             & 0,01                             \\
Kunststoff-Vinyl  & 3mm             & 0,015                            \\
Laminat           & 9mm             & 0,044                            \\
Linoleum          & 2,5mm           & 0,015                            \\
Marmorplatten     & 30mm            & 0,014                            \\
Mosaikparkett     & 8mm             & 0,038                            \\
Naturstein        & 20mm            & 0,017                            \\
Nadelvlies        & 6,5mm           & 0,12                             \\
Stabparkett       & 22mm            & 0,105                            \\
Teppich           & 8mm             & 0,1                             
\end{tabular}
\end{table}

Quellen: 
- https://www.selfio.de/heizung/fussbodenheizung/ratgeber/aufbauhoehe/
- https://www.warmupdeutschland.de/blog/bodenbelaege-fuer-fussbodenheizung-wie-gut-eignen-sich-vinyl-fliesen-oder-teppich
- https://www.heizsparer.de/heizung/heizkorper/fussbodenheizung/passender-bodenbelag

## Verwendete Materialien

### Heizrohre
Für die Rohre sind drei Materialien verbreitet:

- Polyethylen PE-X
- PE-X mit Aluminium-Zwischenschicht
- Kupfer

"Im oder unter dem Heizestrich werden Rohre aus überwiegend Kunststoff oder seltener aus Kupfer verlegt. Der am meisten verwendete Kunststoff ist das vernetzte, sauerstoffdichte Polyethylen (PE-X), oft mit zusätzlicher Aluminium-Zwischenschicht."

Quelle: https://www.heizsparer.de/heizung/heizkorper/fussbodenheizung/funktionsweise

### Bodenbeläge

- Fliesen-, Stein- und polierte Estrichböden
- Holz- und Dielenböden
- Laminatböden
- Vinyl-Böden
- Teppichböden
- Gummiböden

Quelle: https://www.warmupdeutschland.de/blog/bodenbelaege-fuer-fussbodenheizung-wie-gut-eignen-sich-vinyl-fliesen-oder-teppich


## Raumtemperaturen

Für die Wahl der richtigen Raumtemperatur existieren diverse Vorschläge und Richtlinien. Zum Beispiel vom Umweltbundesamt:

- Wohnzimmer: 20 bis 23 °C 
- Schlafzimmer: 17 bis maximal 20 °C
- Kinderzimmer: 20 bis 22 °C, nachts 18 °C
- Küche: 18 °C
- Esszimmer: 20 °C
- Badezimmer: 23 °C
- Büro: 20 bis 22 °C

Quelle: https://www.umweltbundesamt.de/umwelttipps-fuer-den-alltag/heizen-bauen/heizen-raumtemperatur#gewusst-wie

Im Zusammenhang der drohenden Gasknappheit werden jedoch auch kältere Mindestemperaturen diskutiert:

- Tagsüber: 18 °C
- Nachts: 16 °C

Quelle: https://www.tagesschau.de/wirtschaft/verbraucher/gas-sparen-heizen-101.html

Quelle: https://www.heizsparer.de/heizung/heizkorper/fussbodenheizung/fussbodenheizung-planen-heizkreise-berechnen
"Nach DIN EN 1264 gibt es Grenzen in Bezug auf die Oberflächentemperatur bei Betrieb einer Fußbodenheizung. Diese liegt für eine Aufenthaltszone bei 29°C. Das Verhältnis von 29°C Oberflächentemperatur zu 20°C Raumtemperatur ergibt ziemlich genau eine Heizleistung von 100 Watt pro Quadratmeter. Dabei spielt es keine Rolle, ob Kupferrohr oder PE-Rohr für die Fußbodenheizung verwendet wurde. Es ist auch egal, welchen Wärmeerzeuger man verwendet – die Begrenzung der Oberflächentemperatur ist fix.

Moderne Häuser benötigen die Heizleistung von 100 W/m² in der Regel nicht, sondern kommen mit etwa 40 W/m² aus. Dann betragen die Oberflächentemperaturen im Auslegungsfall, also bei -12°C Außentemperatur nur noch 24°C"

Als mögliche Temperaturen können somit vorerst Temperaturen von 16 bis 23 °C untersucht werden:

In [3]:
room_temperature_list = np.arange(16, 23+1, 1).tolist()

room_temperatures = {'minimum_temp_night' : 16,
                     'minimum_temp_day'   : 18,
                     'living_room_min'    : 20,
                     'living_room_max'    : 23,
}

with open('room_temperatures.json', 'w') as file:
    print(json.dumps(room_temperatures))
    json.dump(room_temperatures, file, ensure_ascii=False)


{"minimum_temp_night": 16, "minimum_temp_day": 18, "living_room_min": 20, "living_room_max": 23}
